### Project 02: Create a GUI Notebook Program

Jillian Conway March 25th 2024

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [25]:
# jillian conway comment for lab 06
#Kelly A lab 06 comment
# imports
import tkinter as tk
from tkinter import ttk
import datetime # one module for working with dates and times
import os
import json

# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        self.geometry("600x400") # set the default window size
        self.title('Notebook') #set the default window title
        self.notebook = [] # initialize an attribute named 'notebook' as an empty list
        self.notes = []   # initialize an attribute named 'note' as an empty list
        self.buttons() # calls the button method

    def buttons(self): # this method create the main three buttons that are displayed on the first window
        new_note_button = tk.Button(self, text='new note', command=self.new_note) # new note button, calls the new note function when pressed
        new_note_button.grid(padx=10, pady=10, row=5, column=3, sticky='w') # add to the grid
        
        open_notebook_button = tk.Button(self, text='open notebook', command=self.open_notebook) # open notebook button, calls the open notebook function when pressed
        open_notebook_button.grid(padx=10, pady=10, row=5, column=1, sticky='w') # add to the grid

        save_notebook_button = tk.Button(self, text='save notebook', command=self.save_notebook) # save notebook button, calls the save notebook function when pressed
        save_notebook_button.grid(padx=10, pady=10, row=5, column=0) # add to the grid
        
    def new_note(self): # called when new note button is pressed
        note_window = NoteForm(self, self.notebook, self.notes,title=None, text=None, meta=None, link=None, tags=None) # calls NoteForm sublcass to create the new window, sets the widgets to none
        note_window.create_widgets() # creates the widgets that are created when the new window opens - they will be empty

    def open_notebook(self):
        self.notebook = []
        folder_path = "C:\\Users\\jbcon\\OneDrive\\Desktop\\INST326\\notebook\\Notes" # defines the folder path to open
        row_num = 15 # creates the column for the buttons to be displayed
        if os.path.exists(folder_path): # if there is a folder path
            with open (folder_path, "r") as jsonfile: # open json file in read mode
                new_notes_json = json.load(jsonfile) # load the json data into a new note
                for note_data in new_notes_json: # for each note in the loaded data
                    # creates a note object using MakeNote
                    note = MakeNote({"title": note_data.get("title", ""), "text": note_data.get("text", ""), "meta": note_data.get("meta", ""),"link": note_data.get("link", ""),"tags": note_data.get("tags", [])})
                    self.notebook.append(note)  # Add note to self.notebook
                    # creates a button for each note - calls open_note when pressed
                    button_for_note = tk.Button(self, text=note_data.get("title", ""), command=lambda display_note=note: self.open_note(display_note))
                    button_for_note.grid(padx=10, pady=10, row=row_num, column=0, sticky='w') # adds it to the grid
                    row_num += 1 # incriments the column for each note to be displayed
        else:
            print("there is no folder path available") # if there is an error about finding folder path
                        
    def open_note(self,note):
        # when the button for the note is pressed
        # creates a note window and assigns the information to the widgets
        note_window = NoteForm(self, self.notebook, self.notes, title=note.title, text=note.text, meta=note.meta, link=note.link, tags=note.tags)
        note_window.create_widgets_with_values() # adds the note information to the window
        

    def save_notebook(self):
    # convert notebook to json notebook
        notebook_json = []
        for note in self.notebook:
            #convert make note object to json and add to notebook_json
            notebook_json.append(note.note_dict)
        filepath = "C:\\Users\\jbcon\\OneDrive\\Desktop\\INST326\\notebook\\Notes"
        with open(filepath, "w") as jsonfile: # open jsonfile in write mode
            json.dump(notebook_json,jsonfile) # write notebook to the json file
        
       

# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example
# the submit method calls the MakeNote class that transforms the the entered data into a new note object.

class NoteForm(tk.Toplevel):
    
    def __init__(self, master, notebook, notes, title, text, meta, link, tags): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window
        self.notebook = notebook
        self.notes = notes
        self.note_title = title
        self.note_text = text
        self.note_link = link
        self.note_tags = tags
        self.note_meta = meta

    def create_widgets_with_values(self): # called when you are opening the note already created
        
        # create our note title label
        title_label = tk.Label(self, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')
        
        # create our note text label
        text_label = tk.Label(self, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')

        # create our note link label
        link_label = tk.Label(self, bg='light gray', text='Note link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')

        # create our note tags label
        tag_label = tk.Label(self, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')

        # create our note meta label
        meta_label = tk.Label(self, bg= "light gray", text = "Meta:")
        meta_label.grid(padx=10,pady=10,row = 5,column = 0,sticky="e")

        # create label with the note title that was submited and saved
        self.note_title_label = tk.Label(self, width=80, text=self.note_title)
        self.note_title_label.grid(padx=10, pady=10, row=1, column=1, sticky='w')

         # create text box with the note text that was submited and saved
        self.note_text_box = tk.Text(self, height=10, width=60)
        self.note_text_box.insert('1.0', self.note_text)
        self.note_text_box.grid(padx=10, pady=10, row=2, column=1)

        # create label with the note link that was submited and saved
        self.note_link_label = tk.Label(self, width=80, text=self.note_link)
        self.note_link_label.grid(padx=10, pady=10, row=3, column=1, sticky='w')

        # create label with the note tags that was submited and saved
        self.note_tags_label = tk.Label(self, width=80, text=self.note_tags)
        self.note_tags_label.grid(padx=10, pady=10, row=4, column=1, sticky='w')

        # create label with the note meta that was submited and saved
        self.note_meta_box = tk.Text(self, height=5,width=60)
        self.note_meta_box.insert("1.0", self.note_meta)
        self.note_meta_box.grid(padx=10, pady=10, row=5, column=1)

    def create_widgets(self): # called when user wants to enter a new note
        #create the labels
        title_label = tk.Label(self, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')

        text_label = tk.Label(self, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')

        link_label = tk.Label(self, bg='light gray', text='Note link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')

        tag_label = tk.Label(self, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')

        # create our note title entry field
        self.note_title = tk.Entry(self, width=80)
        self.note_title.grid(padx=10, pady=10, row=1, column=1, sticky='w')
        self.note_title.insert(0, 'New note title') # adds default text (useful during development)

        # create our note text field
        self.note_text = tk.Text(self, height=10, width=60)
        self.note_text.grid(padx=10, pady=10, row=2, column=1)
        self.note_text.insert('1.0', "Enter your text") # adds default text (useful during development)

        # create our note link entry field
        self.note_link = tk.Entry(self, width=80)
        self.note_link.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        self.note_link.insert(0, 'If there is a link with this note enter it here. Else, enter "None"') # adds default text (useful during development)

        # create our note link entry field
        self.note_tags = tk.Entry(self, width=80)
        self.note_tags.grid(padx=10, pady=10, row=4, column=1, sticky='w')
        self.note_tags.insert(0, 'Enter #tags, separated by commas') # adds default text (useful during development)
        submit_button = tk.Button(self, text='submit', command=self.submit)
        submit_button.grid(padx=10, pady=10, row=6, column=1, sticky='w')

        
    def submit(self):
        note_dict = {} # initializes and empty dictionary 
        note_dict["title"] = self.note_title.get() # gets the name of the title and adds it
        note_dict["tags"] = self.note_tags.get() # gets the tags and adds it to dictionary
        note_dict["link"] = self.note_link.get() # gets the link and adds it
        note_dict["text"] = self.note_text.get("1.0",'end') # gets the text and adds it
        # caluclate date, time and timezone
        now = datetime.datetime.now() # gets the current date and time
        local_now = now.astimezone() # shows the local time and the GMT offset
        local_tz = local_now.tzinfo 
        note_dict["meta"] = str(now) , " ", str(local_now) , " " , str(local_tz) # creates a string of the meta data and adds it to the dictionary
        new_note = MakeNote(note_dict) # creates a note using the dictionary
        self.notebook.append(new_note) # adds the dictionary to the notebook


    
# The MakeNote class takes a dictionary containing the data entered into the form window,
# and transforms it into a new note object.
# At present the note objects have attributes but no methods.

class MakeNote():
    def __init__(self, note_dict):
        # tajes a dictionary and creates a new note object
        self.tags = note_dict["tags"]
        self.title = note_dict["title"]
        self.text = note_dict["text"]
        self.link = note_dict["link"]
        self.meta = note_dict["meta"]
        self.note_dict = note_dict

   


# main execution

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop()

#### Print your three notes below

# print your notes here
Title: opening/close files in python

Text: To open and close a file in python, use the open() function to initiate file operations and specify the file name and access mode. Access modes include read-only (“r”), read and write(“r+”), write- only (“w”), write and read (w+), append-only (“a”), and append and read (“a+”). The file should exist in the same directory as the Python script unless specifying the full file path. After, make sure to close the file using the close() method to prevent unintended modifications. 

Link: https://www.geeksforgeeks.org/how-to-open-and-close-a-file-in-python/

Tags: #openFiles #pythonIO #FileIO


Title: using dictionaries 

Text: Dictionaries in Python store data in key-value pairs, providing an ordered and changeable collection. Dictionaries maintain the order of insertion, ensuring consistency in iteration. Additionally, dictionaries do not allow duplicate keys, with duplicate key assignments resulting in overwriting existing values.

Link: https://www.w3schools.com/python/python_dictionaries.asp

Tags: #python #pythonDictionary #DataStructures


Title: Structuring a repository 

Text:  To structure a Python repository effectively, organize it with folders for documentation, source code, tests, and examples. Utilize setup files like setup.py and setup.cfg to define installation instructions and metadata for the package. Include a requirements.txt file listing development dependencies, a README.rst file for project information, and a .gitignore file to specify which files Git should ignore. Additionally, configure a tox.ini file for running unit tests across multiple Python versions using tox, ensuring code compatibility and reliability.

Link: https://github.com/yngvem/python-project-structure?tab=readme-ov-file#structuring-a-repository

Tags: #GitHub #Repo #Python
